# Save fastf1 data a to SQL database

Useful for generating quick reports during the race.


In [1]:
import fastf1

GRAND_PRIX = 'Spain'
YEAR = 2024
DRIVER = 'HAM'
SESSION = 'FP1'

session = fastf1.get_session(YEAR, GRAND_PRIX, SESSION)
session.load(weather=False, messages=False)

req         WARNING 	DEFAULT CACHE ENABLED! (1.04 GB) /Users/diegomaradona/Library/Caches/fastf1
core           INFO 	Loading data for Spanish Grand Prix - Practice 1 [v3.3.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	No lap data for driver 27
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 27)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 21 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '22', '23', '24', '27', '3', '31', '4', '44', '50', '5

In [2]:
df = session.laps.pick_team('Mercedes')
df = df[df['Deleted'].isna()]
df['LapNumber'] = df['LapNumber'].astype(int)
df = df.sort_values(by='LapNumber')
print(f"df len: {len(df)}")
df.head()

df len: 59


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
368,0 days 00:16:01.111000,HAM,44,NaT,1,1.0,0 days 00:14:15.834000,NaT,NaT,0 days 00:00:35.817000,...,True,Mercedes,0 days 00:14:15.834000,2024-06-21 11:30:16.280,1,NaN,None,,False,False
456,0 days 00:15:55.553000,RUS,63,NaT,1,1.0,0 days 00:14:18.199000,NaT,NaT,0 days 00:00:37.699000,...,True,Mercedes,0 days 00:14:18.199000,2024-06-21 11:30:18.645,1,NaN,None,,False,False
457,0 days 00:17:12.577000,RUS,63,0 days 00:01:17.024000,2,1.0,NaT,NaT,0 days 00:00:22.713000,0 days 00:00:30.820000,...,True,Mercedes,0 days 00:15:55.553000,2024-06-21 11:31:55.999,1,NaN,None,,False,True
369,0 days 00:17:18.743000,HAM,44,0 days 00:01:17.632000,2,1.0,NaT,NaT,0 days 00:00:22.815000,0 days 00:00:31.375000,...,True,Mercedes,0 days 00:16:01.111000,2024-06-21 11:32:01.557,1,NaN,None,,False,True
458,0 days 00:19:24.680000,RUS,63,0 days 00:02:12.103000,3,1.0,NaT,NaT,0 days 00:00:38.542000,0 days 00:00:57.766000,...,True,Mercedes,0 days 00:17:12.577000,2024-06-21 11:33:13.023,1,NaN,None,,False,True


#### Convert time columns to seconds


In [3]:
time_columns = ['LapTime', 'PitOutTime', 'PitInTime',
                'Sector1Time', 'Sector2Time', 'Sector3Time', 'LapStartTime']
for col in time_columns:
    if col in df.columns:
        df[col + 'Seconds'] = df[col].dt.total_seconds()
        df.drop(columns=[col], inplace=True, axis=1)
        print(f"Converted {col} to seconds.")

df.drop(columns=[col for col in ['Sector1SessionTime', 'Sector2SessionTime',
        'Sector3SessionTime'] if col in df.columns], inplace=True, axis=1)
df.columns

Converted LapTime to seconds.
Converted PitOutTime to seconds.
Converted PitInTime to seconds.
Converted Sector1Time to seconds.
Converted Sector2Time to seconds.
Converted Sector3Time to seconds.
Converted LapStartTime to seconds.


Index(['Time', 'Driver', 'DriverNumber', 'LapNumber', 'Stint', 'SpeedI1',
       'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest', 'Compound',
       'TyreLife', 'FreshTyre', 'Team', 'LapStartDate', 'TrackStatus',
       'Position', 'Deleted', 'DeletedReason', 'FastF1Generated', 'IsAccurate',
       'LapTimeSeconds', 'PitOutTimeSeconds', 'PitInTimeSeconds',
       'Sector1TimeSeconds', 'Sector2TimeSeconds', 'Sector3TimeSeconds',
       'LapStartTimeSeconds'],
      dtype='object')

In [4]:
df['TimeInMinutes'] = df['Time'].dt.total_seconds() / 60
df.drop(columns=['Time'], inplace=True, axis=1)
df

,Driver,DriverNumber,LapNumber,Stint,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,...,FastF1Generated,IsAccurate,LapTimeSeconds,PitOutTimeSeconds,PitInTimeSeconds,Sector1TimeSeconds,Sector2TimeSeconds,Sector3TimeSeconds,LapStartTimeSeconds,TimeInMinutes
368,HAM,44,1,1.0,215.0,220.0,281.0,207.0,False,HARD,...,False,False,NaN,855.834,NaN,NaN,35.817,24.869,855.834,16.018517
456,RUS,63,1,1.0,208.0,79.0,282.0,241.0,False,HARD,...,False,False,NaN,858.199,NaN,NaN,37.699,33.034,858.199,15.925883
457,RUS,63,2,1.0,275.0,292.0,285.0,316.0,True,HARD,...,False,True,77.024,NaN,NaN,22.713,30.820,23.491,955.553,17.209617
369,HAM,44,2,1.0,287.0,287.0,282.0,316.0,True,HARD,...,False,True,77.632,NaN,NaN,22.815,31.375,23.442,961.111,17.312383
458,RUS,63,3,1.0,128.0,152.0,280.0,258.0,False,HARD,...,False,True,132.103,NaN,NaN,38.542,57.766,35.795,1032.577,19.411333
370,HAM,44,3,1.0,124.0,127.0,NaN,252.0,False,HARD,...,False,False,138.480,NaN,1174.540,34.776,60.313,43.391,1038.743,19.620383
459,RUS,63,4,1.0,282.0,291.0,282.0,318.0,True,HARD,...,False,True,75.868,NaN,NaN,22.485,30.537,22.846,1164.680,20.675800
371,HAM,44,4,2.0,162.0,209.0,280.0,149.0,False,HARD,...,False,False,138.969,1202.155,NaN,60.718,46.940,31.311,1177.223,21.936533
460,RUS,63,5,1.0,178.0,220.0,NaN,276.0,False,HARD,...,False,False,114.638,NaN,1352.485,32.708,46.244,35.686,1240.548,22.586433
372,HAM,44,5,2.0,288.0,301.0,280.0,318.0,True,HARD,...,False,True,76.650,NaN,NaN,22.563,30.890,23.197,1316.192,23.214033


In [5]:
df

,Driver,DriverNumber,LapNumber,Stint,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,...,FastF1Generated,IsAccurate,LapTimeSeconds,PitOutTimeSeconds,PitInTimeSeconds,Sector1TimeSeconds,Sector2TimeSeconds,Sector3TimeSeconds,LapStartTimeSeconds,TimeInMinutes
368,HAM,44,1,1.0,215.0,220.0,281.0,207.0,False,HARD,...,False,False,NaN,855.834,NaN,NaN,35.817,24.869,855.834,16.018517
456,RUS,63,1,1.0,208.0,79.0,282.0,241.0,False,HARD,...,False,False,NaN,858.199,NaN,NaN,37.699,33.034,858.199,15.925883
457,RUS,63,2,1.0,275.0,292.0,285.0,316.0,True,HARD,...,False,True,77.024,NaN,NaN,22.713,30.820,23.491,955.553,17.209617
369,HAM,44,2,1.0,287.0,287.0,282.0,316.0,True,HARD,...,False,True,77.632,NaN,NaN,22.815,31.375,23.442,961.111,17.312383
458,RUS,63,3,1.0,128.0,152.0,280.0,258.0,False,HARD,...,False,True,132.103,NaN,NaN,38.542,57.766,35.795,1032.577,19.411333
370,HAM,44,3,1.0,124.0,127.0,NaN,252.0,False,HARD,...,False,False,138.480,NaN,1174.540,34.776,60.313,43.391,1038.743,19.620383
459,RUS,63,4,1.0,282.0,291.0,282.0,318.0,True,HARD,...,False,True,75.868,NaN,NaN,22.485,30.537,22.846,1164.680,20.675800
371,HAM,44,4,2.0,162.0,209.0,280.0,149.0,False,HARD,...,False,False,138.969,1202.155,NaN,60.718,46.940,31.311,1177.223,21.936533
460,RUS,63,5,1.0,178.0,220.0,NaN,276.0,False,HARD,...,False,False,114.638,NaN,1352.485,32.708,46.244,35.686,1240.548,22.586433
372,HAM,44,5,2.0,288.0,301.0,280.0,318.0,True,HARD,...,False,True,76.650,NaN,NaN,22.563,30.890,23.197,1316.192,23.214033


## Document Loading

https://python.langchain.com/v0.1/docs/use_cases/sql/


In [11]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///laps.db')

df.to_sql(name="laps", con=engine)

59

In [12]:
from sqlalchemy import text
with engine.connect() as conn:
    res = conn.execute(text("SELECT * FROM laps")).fetchall()

    print(res)

[(368, 'HAM', '44', 1, 1.0, 215.0, 220.0, 281.0, 207.0, 0, 'HARD', 1.0, 1, 'Mercedes', '2024-06-21 11:30:16.280000', '1', None, None, '', 0, 0, None, 855.834, None, None, 35.817, 24.869, 855.834, 16.018516666666667), (456, 'RUS', '63', 1, 1.0, 208.0, 79.0, 282.0, 241.0, 0, 'HARD', 1.0, 1, 'Mercedes', '2024-06-21 11:30:18.645000', '1', None, None, '', 0, 0, None, 858.199, None, None, 37.699, 33.034, 858.199, 15.925883333333333), (457, 'RUS', '63', 2, 1.0, 275.0, 292.0, 285.0, 316.0, 1, 'HARD', 2.0, 1, 'Mercedes', '2024-06-21 11:31:55.999000', '1', None, None, '', 0, 1, 77.024, None, None, 22.713, 30.82, 23.491, 955.553, 17.209616666666665), (369, 'HAM', '44', 2, 1.0, 287.0, 287.0, 282.0, 316.0, 1, 'HARD', 2.0, 1, 'Mercedes', '2024-06-21 11:32:01.557000', '1', None, None, '', 0, 1, 77.632, None, None, 22.815, 31.375, 23.442, 961.111, 17.312383333333333), (458, 'RUS', '63', 3, 1.0, 128.0, 152.0, 280.0, 258.0, 0, 'HARD', 3.0, 1, 'Mercedes', '2024-06-21 11:33:13.023000', '1', None, None, ''